<a href="https://colab.research.google.com/github/bksat90/FinalProject/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Installation of KerasCV.
!pip install -q --upgrade keras-cv

In [32]:
# Importing relevant modules.
import os
import tqdm
import numpy as np

os.environ["KERAS_BACKEND"] = "tensorflow"
from tensorflow import data as tf_data
import tensorflow_datasets as tfds
import keras
import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization
import xml.etree.ElementTree as ET
import tensorflow as tf

In [33]:
# Import and mount Google Drive.
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
# # Unzip the dataset file and store it in a folder called images.
# !unzip "/content/drive/MyDrive/Project_BC/data_voc.zip" -d "/content/drive/MyDrive/Project_BC/data/"

In [44]:
class_ids = [
    "0",
    "1",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

# Path to images and annotations
files_path = "/content/drive/MyDrive/Project_BC/data/"

# Get all XML file paths in path_annot and sort them
xml_files = sorted(
    [
        os.path.join(files_path, file_name)
        for file_name in os.listdir(files_path)
        if file_name.endswith(".xml")
    ]
)

# Get all train JPEG image file paths in path_images and sort them
jpg_files = sorted(
    [
        os.path.join(files_path, file_name)
        for file_name in os.listdir(files_path)
        if file_name.endswith(".jpg")
    ]
)

In [45]:
def parse_annotation(xml_file, path_images):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(path_images, image_name)

    boxes = []
    classes = []
    for obj in root.iter("object"):
        cls = obj.find("name").text
        classes.append(cls)

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])

    class_ids = [
        list(class_mapping.keys())[list(class_mapping.values()).index(cls)]
        for cls in classes
    ]
    return image_path, boxes, class_ids

In [46]:
# training set
image_paths = []
bbox = []
classes = []
for xml_file in xml_files:
    image_path, boxes, class_ids = parse_annotation(xml_file, files_path)
    image_paths.append(image_path)
    bbox.append(boxes)
    classes.append(class_ids)